In [86]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
import pandas as pd

In [87]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [88]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("make_vector_DB")

LangSmith 추적을 시작합니다.
[프로젝트명]
make_vector_DB


In [89]:
# 임베딩할 데이터셋 정하기
DATA_PATH="../datas/merge_data/"
DATA_NAME="어트랙션정보병합"
DATA_EXTENTION=".xlsx"

In [90]:


# Excel 파일 읽기
df = pd.read_excel(DATA_PATH + DATA_NAME + DATA_EXTENTION)
df = df.head(100)


In [91]:
model = ChatOpenAI(model_name="gpt-4o", temperature=0)

In [92]:
# 컬럼 데이터
column_data = ', '.join(df.columns)

# 예시 행 데이터
example_row_data = ', '.join(map(str, df.iloc[0].tolist()))


In [93]:
question = \
"""
주어진 Column data을 각 컬럼을 key값, 구어체로 번역된 컬럼을 value값으로 만들어줘
예시 데이터를 줄테니 알맞은 컬럼 이름을 생성해줘.
# 예시
'원래 컬럼': '구어체로 번역된 컬럼'

"""

# JSON 출력 파서 초기화
parser = JsonOutputParser()

# 프롬프트 템플릿을 설정합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 영어로 된 컬럼이름을 한국어로 바꾸어주는 어시스턴트 입니다."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}\n\n#Column data: {column_data}\n\n#Example Column data: {example_column_data}"),
    ]
)

# 지시사항을 프롬프트에 주입합니다.
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# 프롬프트, 모델, 파서를 연결하는 체인 생성
chain = prompt | model | parser

# 체인을 호출하여 쿼리 실행
column_vocab = chain.invoke({"question": question, "column_data": column_data, "example_column_data": example_row_data})

# 출력을 확인합니다.
print(column_vocab)


{'UC_SEQ': '고유번호', 'MAIN_TITLE': '주요제목', 'GUGUN_NM': '구군이름', 'LAT': '위도', 'LNG': '경도', 'PLACE': '장소', 'TITLE': '제목', 'SUBTITLE': '부제목', 'MAIN_PLACE': '주요장소', 'ADDR1': '주소1', 'ADDR2': '주소2', 'CNTCT_TEL': '연락처', 'HOMEPAGE_URL': '홈페이지주소', 'TRFC_INFO': '교통정보', 'USAGE_DAY': '이용일', 'HLDY_INFO': '휴일정보', 'USAGE_DAY_WEEK_AND_TIME': '이용요일및시간', 'USAGE_AMOUNT': '이용금액', 'MIDDLE_SIZE_RM1': '중간크기방1', 'MAIN_IMG_NORMAL': '주요이미지', 'MAIN_IMG_THUMB': '썸네일이미지', 'ITEMCNTNTS': '항목내용'}


In [ ]:
# 문장 데이터에서 제외될 컬럼
exclude_columns = ['combine_column']

# Y, N같은 데이터는 문자열로 변환
replacement_map = {
    "Y": "가능합니다.",
    "N": "불가능합니다.",
}

# 데이터 프레임 행을을 문장으로 변형해서 컬럼 추가.
df["combine_column"] = df.apply(
    lambda row: ' '.join(
        [f"{column_vocab[col]}은 {replacement_map.get(row[col], '알 수 없습니다.' if pd.isna(row[col]) else str(row[col]) + '입니다.')}" for col in df.columns if col not in exclude_columns]
    ),
    axis=1
)

In [95]:
df["combine_column"][0]

'고유번호은 58입니다. 주요제목은 초량이바구길 (한,영,중간,중번,일)입니다. 구군이름은 동구입니다. 위도은 35.11635입니다. 경도은 129.03874입니다. 장소은 초량이바구길, 이바구공작소, 168계단입니다. 제목은 이야기로 피어난 어제의 기억 초량이바구길입니다. 부제목은 이바구 꽃이 피었습니다입니다. 주요장소은 알 수 없습니다. 주소1은 부산광역시 동구 중앙대로 209번길 16입니다. 주소2은 알 수 없습니다. 연락처은 알 수 없습니다. 홈페이지주소은 http://www.2bagu.co.kr입니다. 교통정보은 도시철도 1호선 부산역 7번 출구 도보 2분, 초량역 1번 출구 도보 8분\n버스 26, 27, 40, 41, 59, 81, 87, 103, 1003, 1004 부산역 하차 도보 2분\n주차 인근 공영주차장입니다. 이용일은 알 수 없습니다. 휴일정보은 알 수 없습니다. 이용요일및시간은 알 수 없습니다. 이용금액은 알 수 없습니다. 중간크기방1은 알 수 없습니다. 주요이미지은 https://www.visitbusan.net/uploadImgs/files/cntnts/20240906183754220_ttiel입니다. 썸네일이미지은 https://www.visitbusan.net/uploadImgs/files/cntnts/20240906183754220_thumbL입니다. 항목내용은 부산항을 기준으로 근처에 보이는 산 중턱마다 죄다 빽빽하게 들어서 있는 주택들.\n정든 고향 남겨두고 부산으로 피난 온 사람들이 산으로 올라가 일군 마을. 일감만 있다면 부두로, 역으로, 국제시장으로 하루에도 수십 번을 오르내렸을 168계단.\n경상도 사투리도 모르던 사람들이 피워낸 이야기 길, 초량이바구길을 만나러 가자. \n\n<p class="font-size28 colorDarkBlue medium">초량이바구길</p><p class="font-size20 medium">옛백제병원 - 남선창고터 - 초량교회 - 168계단 -김민부 전망대 - 이바구공작소 - 장기

In [96]:
from langchain_community.document_loaders import DataFrameLoader

# 데이터 프레임 로더 설정, 페이지 내용 컬럼 지정
loader = DataFrameLoader(df, page_content_column="combine_column")

# 문서 로드
docs = loader.load()


In [97]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

# 임베딩
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [98]:
# DB 생성
db = FAISS.from_documents(documents=docs, embedding=OpenAIEmbeddings())

In [99]:
# 로컬 Disk 에 저장
db.save_local(folder_path="faiss_db", index_name=DATA_NAME)